## Imports

In [2]:
import dostools
import importlib
import numpy as np
import pickle
import torch
import sys
import matplotlib.pyplot as plt
torch.set_default_dtype(torch.float64) 
sys.modules['dostools.src'] = dostools

In [3]:
import dostools.datasets.data as data
import dostools.utils.utils as utils

n_structures = 1039
np.random.seed(0)
n_train = int(0.8 * n_structures)
train_index = np.arange(n_structures)
np.random.shuffle(train_index)
test_index = train_index[n_train:]
train_index = train_index[:n_train]

with torch.no_grad():
    structures = data.load_structures(":")
    n_structures = len(structures) #total number of structures
    for structure in structures:#implement periodicity
        structure.wrap(eps = 1e-12) 
    n_atoms = np.zeros(n_structures, dtype = int) #stores number of atoms in each structures
    for i in range(n_structures):
        n_atoms[i] = len(structures[i])

    #eigenergies, emin, emax = dostools.src.datasets.data.load_eigenenergies(unpack = True, n_structures = len(structures))
    xdos = torch.tensor(data.load_xdos())
    ldos = torch.tensor(data.load_ldos())
    ldos *= 2

    print ("ldos shape is {}".format(ldos.shape))
    mean_dos_per_atom = ldos[train_index].mean(axis = 0) #only calculated for train set to prevent data leakage
    print ("mean dos shape is {}".format(mean_dos_per_atom.shape))
    
    
    y_pw = ldos - mean_dos_per_atom
    y_lcdf = torch.cumsum(y_pw, dim = 1)
    _, pc_vectors = utils.build_pc(ldos[train_index], mean_dos_per_atom[None,:], n_pc = 10)
    y_pc = utils.build_coeffs(ldos - mean_dos_per_atom[None,:], pc_vectors)
    Silicon = data.load_features()
    kMM = data.load_kMM()

ldos shape is torch.Size([1039, 778])
mean dos shape is torch.Size([778])
Variance covered with 10 PCs is = 0.9871211778950163


## Evaluator

In [4]:
import dostools.evaluation.evaluation as evaluation
importlib.reload(evaluation)
import dostools.models.training as training
importlib.reload(training)

targets = {
    'pw' : ldos,
    'lcdf' : y_lcdf,
    'pc' : y_pc
}
evaluator = evaluation.Evaluator(targets, xdos, mean_dos_per_atom)

## Analytical Models

In [5]:
import dostools.models.models as models
importlib.reload(models)


Ridge_pw_kernel = models.RidgeRegression(Silicon.Features['structure_avekerneldescriptors'], "avekernels", y_pw, "pw", xdos)
Ridge_pw_kernel.obtain_weights([1.0,0.75,0.5,0.25], train_index)
Ridge_pw_kernel.get_DosRMSE(evaluator, train_index, test_index)
Ridge_pw_kernel.get_targetRMSE(evaluator, train_index, test_index)

Ridge_pw_soap = models.RidgeRegression(Silicon.Features['structure_avedescriptors'], "avesoap", y_pw, "pw", xdos)
Ridge_pw_soap.obtain_weights([1.0,0.75,0.5,0.25], train_index) 
Ridge_pw_soap.get_DosRMSE(evaluator, train_index, test_index) 
Ridge_pw_soap.get_targetRMSE(evaluator, train_index, test_index) 

GPR_pw = models.GPR(Silicon.Features['structure_avekerneldescriptors'], "avekernels", y_pw, "pw", xdos, kMM)
GPR_pw.obtain_weights([1.0,0.75,0.5,0.25], train_index)
GPR_pw.get_DosRMSE(evaluator, train_index, test_index)
GPR_pw.get_targetRMSE(evaluator, train_index, test_index)


The performance at reg value 1.0000000000000031e-10 is 0.0248891107414344
The performance at reg value 1.0000000000000018e-08 is 0.01598743351010591
The performance at reg value 1.0000000000000004e-06 is 0.01472758167349197
The performance at reg value 9.999999999999991e-05 is 0.018910912251443382
The performance at reg value 0.010000000000000005 is 0.027055844562504025
The performance at reg value 1.0 is 0.03807414403136081
The performance at reg value 100.00000000000004 is 0.05733310370242154
The log_grid error is:
[0.02488911 0.01598743 0.01472758 0.01891091 0.02705584 0.03807414
 0.0573331 ]
Init value is: 1.0000000000000004e-06
The performance at reg value 1.0000000000000004e-06 is 0.01472758167349197
The performance at reg value 5.011872336272722e-07 is 0.014678866992409521
The performance at reg value 2.511886431509578e-07 is 0.01476697565423737
The performance at reg value 7.07945784384138e-07 is 0.014682803520751407
Optimal regularization value for Ridge, Feature:avekernels, t

The performance at reg value 0.005011872336272726 is 0.032294919860217396
Optimal regularization value for Ridge, Feature:avesoap, target: pw, train_ratio: 0.25 is 0.005011872336272726
The performance at reg value 1.0000000000000031e-10 is 0.02445598347678289
The performance at reg value 1.0000000000000018e-08 is 0.041981368302301106
The performance at reg value 1.0000000000000004e-06 is 0.09412310200603374
The performance at reg value 9.999999999999991e-05 is 0.0942241889697976
The performance at reg value 0.010000000000000005 is 0.09422419910593646
The performance at reg value 1.0 is 0.0942241991069501
The performance at reg value 100.00000000000004 is 0.0942241991069502
The log_grid error is:
[0.02445598 0.04198137 0.0941231  0.09422419 0.0942242  0.0942242
 0.0942242 ]
Init value is: 1.0000000000000031e-10
The performance at reg value [1.e-10] is 0.024806809158555702
The performance at reg value [3.16227766e-11] is 0.023891115448401216
The performance at reg value [1.e-11] is 0.023

In [101]:
Ridge_lcdf_kernel = models.RidgeRegression(Silicon.Features['structure_avekerneldescriptors'], "avekernels", y_lcdf, "lcdf", xdos)
Ridge_lcdf_kernel.obtain_weights([1.0,0.75,0.5,0.25], train_index)
Ridge_lcdf_kernel.get_DosRMSE(evaluator, train_index, test_index)
Ridge_lcdf_kernel.get_targetRMSE(evaluator, train_index, test_index)

Ridge_lcdf_soap = models.RidgeRegression(Silicon.Features['structure_avedescriptors'], "avesoap", y_lcdf, "lcdf", xdos)
Ridge_lcdf_soap.obtain_weights([1.0,0.75,0.5,0.25], train_index)
Ridge_lcdf_soap.get_DosRMSE(evaluator, train_index, test_index)
Ridge_lcdf_soap.get_targetRMSE(evaluator, train_index, test_index)

GPR_lcdf = models.GPR(Silicon.Features['structure_avekerneldescriptors'], "avekernels", y_lcdf, "lcdf", xdos, kMM)
GPR_lcdf.obtain_weights([1.0,0.75,0.5,0.25], train_index)
GPR_lcdf.get_DosRMSE(evaluator, train_index, test_index)
GPR_lcdf.get_targetRMSE(evaluator, train_index, test_index)

The performance at reg value 9.999999999999996e-11 is 0.30294051089969054
The performance at reg value 9.999999999999982e-09 is 0.24961739104436304
The performance at reg value 1.0000000000000004e-06 is 0.26596574913017423
The performance at reg value 0.00010000000000000009 is 0.3076890689349757
The performance at reg value 0.010000000000000004 is 0.4288955801160411
The performance at reg value 1.0 is 1.0168938487096923
The performance at reg value 100.00000000000004 is 2.0934131122906705
The log_grid error is:
[0.30294051 0.24961739 0.26596575 0.30768907 0.42889558 1.01689385
 2.09341311]
Init value is: 9.999999999999982e-09
The performance at reg value 9.999999999999982e-09 is 0.24961739104436304
The performance at reg value 3.9810717055349665e-09 is 0.2517374519703249
The performance at reg value 2.511886431509575e-08 is 0.2525078253184684
The performance at reg value 6.309573444801934e-09 is 0.2500443950833206
Optimal regularization value for Ridge, Feature:avekernels, target: lcdf

The performance at reg value 9.999999999999996e-11 is 0.3582900919139367
The performance at reg value 9.999999999999982e-09 is 0.37275894760787504
The performance at reg value 1.0000000000000004e-06 is 0.3683979085803729
The performance at reg value 0.00010000000000000009 is 2.628841196282138
The performance at reg value 0.010000000000000004 is 5.318819596087717
The performance at reg value 1.0 is 5.319751341043245
The performance at reg value 100.00000000000004 is 5.319751434245397
The log_grid error is:
[0.35829009 0.37275895 0.36839791 2.6288412  5.3188196  5.31975134
 5.31975143]
Init value is: 9.999999999999996e-11
The performance at reg value [1.e-10] is 0.3617312696251491
The performance at reg value [3.16227766e-11] is 0.37745795705207075
The performance at reg value [3.16227766e-10] is 0.36972685644010284
The performance at reg value [1.77827941e-10] is 0.3745893194886805
The performance at reg value [5.62341325e-11] is 0.36559214078515645
Optimal Regularization value for GPR,

In [ ]:
[369, 341, 745, 521, 278, 5, 193, 37]

## Analytical Evaluation

In [105]:
print ("Ridge_pw_kernel DOS RMSE")
print (Ridge_pw_kernel.DOSrmse)
print ("Ridge_pw_kernel Target RMSE")
print (Ridge_pw_kernel.targetrmse)

Ridge_pw_kernel DOS RMSE
{1.0: array([ 8.94896867, 10.260054  ,  9.21725818,  0.04963029,  0.05958249,
        0.05162265]), 0.75: array([ 9.77312821, 11.12209438, 10.04892532,  0.05420102,  0.06458855,
        0.05628052]), 0.5: array([10.86239665, 11.71137637, 11.03393834,  0.06024202,  0.06801065,
        0.06179724]), 0.25: array([26.12161199, 26.06197913, 26.09796345,  0.14486845,  0.15134789,
        0.14616559])}
Ridge_pw_kernel Target RMSE
{1.0: array([ 8.94896867, 10.260054  ,  9.21725818,  0.04963029,  0.05958249,
        0.05162265]), 0.75: array([ 9.77312821, 11.12209438, 10.04892532,  0.05420102,  0.06458855,
        0.05628052]), 0.5: array([10.86239665, 11.71137637, 11.03393834,  0.06024202,  0.06801065,
        0.06179724]), 0.25: array([26.12161199, 26.06197913, 26.09796345,  0.14486845,  0.15134789,
        0.14616559])}


In [104]:
print ("Ridge_pw_soap DOS RMSE")
print (Ridge_pw_soap.DOSrmse)
print ("Ridge_pw_soap Target RMSE")
print (Ridge_pw_soap.targetrmse)

Ridge_pw_soap DOS RMSE
{1.0: array([10.74889973, 11.6362159 , 10.92844813,  0.05961257,  0.06757417,
        0.06120643]), 0.75: array([11.20180752, 12.09020208, 11.38138437,  0.06212436,  0.07021058,
        0.06374316]), 0.5: array([13.1183806 , 13.29469995, 13.14932045,  0.07275353,  0.07720537,
        0.07364476]), 0.25: array([23.03672075, 22.73477731, 22.9641052 ,  0.12775988,  0.13202607,
        0.12861394])}
Ridge_pw_soap Target RMSE
{1.0: array([10.74889973, 11.6362159 , 10.92844813,  0.05961257,  0.06757417,
        0.06120643]), 0.75: array([11.20180752, 12.09020208, 11.38138437,  0.06212436,  0.07021058,
        0.06374316]), 0.5: array([13.1183806 , 13.29469995, 13.14932045,  0.07275353,  0.07720537,
        0.07364476]), 0.25: array([23.03672075, 22.73477731, 22.9641052 ,  0.12775988,  0.13202607,
        0.12861394])}


In [106]:
print ("GPR_pw DOS RMSE")
print (GPR_pw.DOSrmse)
print ("GPR_pw Target RMSE")
print (GPR_pw.targetrmse)

GPR_pw DOS RMSE
{1.0: array([22.11708717, 21.35773438, 21.94992095,  0.12265967,  0.12402926,
        0.12293385]), 0.75: array([21.5088905 , 20.74365928, 21.20937588,  0.11865888,  0.12046319,
        0.11878631]), 0.5: array([21.06248509, 20.88436152, 21.41177157,  0.11681263,  0.12128028,
        0.11991986]), 0.25: array([23.83315022, 24.49232399, 24.30919548,  0.12613305,  0.14223254,
        0.13614732])}
GPR_pw Target RMSE
{1.0: array([22.11708717, 21.35773438, 21.94992095,  0.12265967,  0.12402926,
        0.12293385]), 0.75: array([21.5088905 , 20.74365928, 21.20937588,  0.11865888,  0.12046319,
        0.11878631]), 0.5: array([21.06248509, 20.88436152, 21.41177157,  0.11681263,  0.12128028,
        0.11991986]), 0.25: array([23.83315022, 24.49232399, 24.30919548,  0.12613305,  0.14223254,
        0.13614732])}


In [109]:
print ("Ridge_lcdf_kernel DOS RMSE")
print (Ridge_lcdf_kernel.DOSrmse)
print ("Ridge_lcdf_kernel Target RMSE")
print (Ridge_lcdf_kernel.targetrmse)

Ridge_lcdf_kernel DOS RMSE
{1.0: array([ 8.21098306, 10.33286287,  8.64789288,  0.04553748,  0.0600053 ,
        0.04843383]), 0.75: array([10.21666806, 11.8486123 , 10.5510134 ,  0.05666086,  0.06880761,
        0.05909255]), 0.5: array([11.17952744, 12.5301001 , 11.45505122,  0.0620008 ,  0.07276516,
        0.06415575]), 0.25: array([15.06591952, 14.74258081, 14.99230181,  0.08355443,  0.08561355,
        0.08396665])}
Ridge_lcdf_kernel Target RMSE
{1.0: array([1.51905341, 2.06275147, 1.63040385, 0.44271821, 0.62213614,
       0.47863633]), 0.75: array([2.19245196, 2.57954095, 2.27138249, 0.63897583, 0.77800242,
       0.66680791]), 0.5: array([2.50723751, 2.78201543, 2.5629707 , 0.73071803, 0.83906974,
       0.75240923]), 0.25: array([3.8251488 , 3.57944404, 3.77342258, 1.11481469, 1.07957818,
       1.1077606 ])}


In [107]:
print ("Ridge_lcdf_soap DOS RMSE")
print (Ridge_lcdf_soap.DOSrmse)
print ("Ridge_lcdf_soap Target RMSE")
print (Ridge_lcdf_soap.targetrmse)

Ridge_lcdf_soap DOS RMSE
{1.0: array([12.5041179 , 13.05041966, 12.61212243,  0.06934688,  0.07578678,
        0.0706361 ]), 0.75: array([13.0034001 , 13.53473352, 13.10808214,  0.07211586,  0.0785993 ,
        0.07341379]), 0.5: array([13.97558488, 14.06515944, 13.9881487 ,  0.07750752,  0.08167961,
        0.07834274]), 0.25: array([16.73216006, 15.43007066, 16.45465778,  0.09279527,  0.08960596,
        0.0921568 ])}
Ridge_lcdf_soap Target RMSE
{1.0: array([3.14754043, 3.16554191, 3.15026236, 0.91733014, 0.95474324,
       0.92481996]), 0.75: array([3.31747847, 3.35290847, 3.32373203, 0.96685748, 1.01125393,
       0.97574531]), 0.5: array([3.67800992, 3.57160966, 3.65498061, 1.07193202, 1.07721529,
       1.07298969]), 0.25: array([4.50041176, 3.95253412, 4.3863266 , 1.31161567, 1.19210401,
       1.28769034])}


In [108]:
print ("GPR_lcdf DOS RMSE")
print (GPR_lcdf.DOSrmse)
print ("GPR_lcdf Target RMSE")
print (GPR_lcdf.targetrmse)

GPR_lcdf DOS RMSE
{1.0: array([20.90363469, 20.46987494, 20.80457749,  0.11592995,  0.11887326,
        0.11651918]), 0.75: array([22.40731733, 22.02863113, 22.25118271,  0.12361527,  0.12792531,
        0.12462111]), 0.5: array([22.55960419, 22.07145294, 22.64809633,  0.12511565,  0.12817399,
        0.12684409]), 0.25: array([23.69955108, 22.92802315, 23.60884309,  0.125426  ,  0.13314829,
        0.13222489])}
GPR_lcdf Target RMSE
{1.0: array([5.63197171, 5.46999487, 5.59690318, 1.64140145, 1.64977775,
       1.64307832]), 0.75: array([6.08147253, 5.83454171, 5.94154172, 1.75597532, 1.75972689,
       1.74425358]), 0.5: array([6.05751721, 6.14648197, 6.31087866, 1.79060255, 1.85380963,
       1.85267952]), 0.25: array([6.62111657, 6.19329568, 6.35071534, 1.75255165, 1.86792888,
       1.86437434])}


## Torch Model

In [56]:
import dostools.models.models as models
import dostools.models.training as training
import dostools.models.architectures as architecture
import dostools.loss.loss as loss
importlib.reload(models)
importlib.reload(training)
importlib.reload(architecture)
importlib.reload(loss)

TLReg_pw_soap = models.TRegression(Silicon.Features['structure_avedescriptors'].double(), "avedesc", y_pw.double(), "pw", "pw", xdos, "Adam", "cpu")
TLReg_pw_kernel = models.TRegression(Silicon.Features['structure_avekerneldescriptors'].double(), "avekerneldesc", y_pw.double(), "pw", "pw", xdos, "Adam", "cpu")
TLReg_lcdf_soap = models.TRegression(Silicon.Features['structure_avedescriptors'].double(), "avedesc", y_lcdf.double(), "lcdf", "pw", xdos, "Adam", "cpu")
TLReg_lcdf_kernel = models.TRegression(Silicon.Features['structure_avekerneldescriptors'].double(), "avekerneldesc", y_lcdf.double(), "lcdf", "pw", xdos, "Adam", "cpu")


In [57]:
torch.manual_seed(0)
TLReg_pw_soap.obtain_weights([1.0], train_index, None, 0.1, 2000000, loss.t_get_mse)
torch.manual_seed(0)
TLReg_pw_kernel.obtain_weights([1.0], train_index, None, 0.001, 2000000, loss.t_get_mse)
torch.manual_seed(0)
TLReg_lcdf_soap.obtain_weights([1.0], train_index, None, 1, 2000000, loss.t_get_mse)
torch.manual_seed(0)
TLReg_lcdf_kernel.obtain_weights([1.0], train_index, None, 0.01, 2000000, loss.t_get_mse)

Currently trying regularization :1.0000000000000031e-10


Epoch: 4898:   0%|▏                                                                                                | 4898/2000000 [01:46<12:02:41, 46.01it/s, lowest_loss=0.0146, pred_loss=0.000213, trigger=1300]


Implemented early stopping with lowest_loss: 0.01460984729623029


Epoch: 4895:   0%|▏                                                                                                | 4895/2000000 [01:45<11:54:11, 46.56it/s, lowest_loss=0.0147, pred_loss=0.000216, trigger=1300]


Implemented early stopping with lowest_loss: 0.014700051137602775
The performance at reg value:1.0000000000000031e-10 is 0.00021480600769089138
Currently trying regularization :1.0000000000000018e-08


...

Implemented early stopping with lowest_loss: 1.033612076020491


Epoch: 14579:   1%|▋                                                                                                    | 14579/2000000 [06:58<15:49:58, 34.83it/s, lowest_loss=1.03, pred_loss=1.06, trigger=1300]


Implemented early stopping with lowest_loss: 1.0296956071502261
The performance at reg value:[6.30957344e-08] is 1.0643871558180287
Optimal regularization value for Ridge, Feature:avekerneldesc, target: lcdf is 6.309573444801918e-08
Currently training torch linear model with features :avekerneldesc, target: lcdf, opt: Adam, lr:0.01


Epoch: 14693:   1%|▋                                                                                                    | 14693/2000000 [07:03<15:53:31, 34.70it/s, lowest_loss=1.04, pred_loss=1.07, trigger=1300]

Implemented early stopping with lowest_loss: 1.0366479699046682


In [70]:
Ridge_pw = models.RidgeRegression(Silicon.Features['structure_avedescriptors'], "avedesc", y_pw, "pw", xdos)
Ridge_pw.obtain_weights([1.0,0.5], train_index)
Ridge_pw.get_DosRMSE(evaluator, train_index, test_index)
Ridge_pw.get_targetRMSE(evaluator, train_index, test_index)


The performance at reg value 9.999999999999996e-11 is 0.0470273564726883
The performance at reg value 9.999999999999982e-09 is 0.03135393777242554
The performance at reg value 1.0000000000000004e-06 is 0.020054425087597604
The performance at reg value 0.00010000000000000009 is 0.016362072888277648
The performance at reg value 0.010000000000000004 is 0.02395674318210591
The performance at reg value 1.0 is 0.04281414485376572
The performance at reg value 100.00000000000004 is 0.08142463483743637
The log_grid error is:
[0.04702736 0.03135394 0.02005443 0.01636207 0.02395674 0.04281414
 0.08142463]
Init value is: 0.00010000000000000009
The performance at reg value 0.00010000000000000009 is 0.016362072888277648
The performance at reg value 6.309573444801939e-05 is 0.016439094014605245
The performance at reg value 0.00015848931924611147 is 0.016417027215670447
The performance at reg value 0.00012589254117941672 is 0.01637212888920042
Optimal regularization value for Ridge, Feature:avedesc, t

In [58]:
TLReg_pw_soap.get_DosRMSE(evaluator, train_index, test_index) 
TLReg_pw_kernel.get_DosRMSE(evaluator, train_index, test_index) 
TLReg_lcdf_soap.get_DosRMSE(evaluator, train_index, test_index) 
TLReg_lcdf_kernel.get_DosRMSE(evaluator, train_index, test_index)
TLReg_pw_soap.get_targetRMSE(evaluator, train_index, test_index) 
TLReg_pw_kernel.get_targetRMSE(evaluator, train_index, test_index) 
TLReg_lcdf_soap.get_targetRMSE(evaluator, train_index, test_index) 
TLReg_lcdf_kernel.get_targetRMSE(evaluator, train_index, test_index)

In [59]:
print ("soap pw")
print (TLReg_pw_soap.models[1.0].reg)
print (TLReg_pw_soap.DOSrmse)
print (TLReg_pw_soap.targetrmse)

soap pw
tensor(3.1623e-12)
{1.0: array([13.67663314, 13.834722  , 13.7035479 ,  0.07584956,  0.0803414 ,
        0.07674879])}
{1.0: array([13.67663314, 13.834722  , 13.7035479 ,  0.07584956,  0.0803414 ,
        0.07674879])}


In [60]:
print ("soap lcdf")
print (TLReg_lcdf_soap.models[1.0].reg)
print (TLReg_lcdf_soap.DOSrmse)
print (TLReg_lcdf_soap.targetrmse) 

soap lcdf
tensor(3.1623e-12)
{1.0: array([15.62364639, 15.66332171, 15.62514124,  0.08664754,  0.0909605 ,
        0.08751096])}
{1.0: array([3.8455992 , 3.77437849, 3.82975315, 1.12077482, 1.13837139,
       1.12429752])}


In [61]:
print ("kernel pw ")
print (TLReg_pw_kernel.models[1.0].reg)
print (TLReg_pw_kernel.DOSrmse)
print (TLReg_pw_kernel.targetrmse)

kernel pw 
tensor(6.3096e-10)
{1.0: array([42.99324575, 42.21080844, 42.81228073,  0.23843724,  0.24512784,
        0.23977665])}
{1.0: array([42.99324575, 42.21080844, 42.81228073,  0.23843724,  0.24512784,
        0.23977665])}


In [62]:
print ("kernel lcdf ")
print (TLReg_lcdf_kernel.models[1.0].reg)
print (TLReg_lcdf_kernel.DOSrmse)
print (TLReg_lcdf_kernel.targetrmse)

kernel lcdf 
tensor(6.3096e-08)
{1.0: array([38.80565144, 37.57889253, 38.53426242,  0.21521316,  0.21822925,
        0.21581696])}
{1.0: array([17.80732676, 16.09720864, 17.45005574,  5.18982932,  4.85499845,
        5.12279869])}


Multiplying loss by col A : Train error col B
1e0 = 16.188
1e3 = 11.73
1e5 = 11.79
1e7 = 11.41
1e10 = 11.66
1e20 = 11.27
1e90 = 11.29

{1.0: array([11.41705718, 12.11622485, 11.5572621 ,  0.06331812,  0.0703617 ,
         0.06472819])}

In [28]:
with torch.no_grad():
    GPR_pred = Silicon.Features['structure_avekerneldescriptors'] @ GPR_pw.weights[1.0]
    Ridge_pred = Ridge_pw.models[1.0].predict(Silicon.Features['structure_avekerneldescriptors'])
    torch_pred = TLReg.models[1.0](Silicon.Features['structure_avedescriptors'].double())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1039x448 and 1000x778)

In [ ]:
plt.plot(xdos, GPR_pred[369], color = 'r', label = "GPR_pred")
plt.plot(xdos, Ridge_pred[369], color = 'g', label = "Ridge_pred")
plt.plot(xdos, torch_pred[369], color = 'c', label = "Torch_pred")
plt.plot(xdos, y_pw[369], color = 'b', label = 'true')
plt.legend()

